In [1]:
library('dplyr')
library('ggplot2')


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [6]:
df = read.csv('titanic.csv', sep=',', dec='.')

In [7]:
head(df, 5)

X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1st,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,Southampton,2,NA,"St Louis, MO"
2,1st,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,11,NA,"Montreal, PQ / Chesterville, ON"
3,1st,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON"
4,1st,0,"Allison, Mr. Hudson Joshua Crei",male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,,135,"Montreal, PQ / Chesterville, ON"
5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON"


## 1.1 Изучаем таблицу

Можно посмотреть на первые пять строк!

In [6]:
head(df,5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,,S


Можно посмотреть на последние пять строк!

In [7]:
tail(df,5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
887,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.00,,S
888,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S
889,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NA,1,2,W./C. 6607,23.45,,S
890,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C
891,891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,,Q


Можно посмотреть на число колонок, строк и размерность таблицы.

In [8]:
ncol(df)  # число колонок
nrow(df)  # число строк
dim(df)   # размерность таблицы

[1] 15

[1] 1309

[1] 1309   15

Можно бросить взор на нашу таблицу!

In [9]:
glimpse(df)

Observations: 1,309
Variables: 15
$ X         <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...
$ pclass    <fctr> 1st, 1st, 1st, 1st, 1st, 1st, 1st, 1st, 1st, 1st, 1st, 1...
$ survived  <int> 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, ...
$ name      <fctr> Allen, Miss. Elisabeth Walton, Allison, Master. Hudson T...
$ sex       <fctr> female, male, female, male, female, male, female, male, ...
$ age       <dbl> 29.0000, 0.9167, 2.0000, 30.0000, 25.0000, 48.0000, 63.00...
$ sibsp     <int> 0, 1, 1, 1, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...
$ parch     <int> 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...
$ ticket    <fctr> 24160, 113781, 113781, 113781, 113781, 19952, 13502, 112...
$ fare      <dbl> 211.3375, 151.5500, 151.5500, 151.5500, 151.5500, 26.5500...
$ cabin     <fctr> B5, C22 C26, C22 C26, C22 C26, C22 C26, E12, D7, A36, C1...
$ embarked  <fctr> Southampton, Southampton, Southampton, Southampton, Sout...
$ boat      <fctr>

Можно посмотреть по таблице основные характеристики.

In [16]:
summary(df[c('Sex', 'Age', 'Pclass', 'SibSp', 'Ticket') ])

     Sex           Age            Pclass          SibSp            Ticket   
 female:314   Min.   : 0.42   Min.   :1.000   Min.   :0.000   1601    :  7  
 male  :577   1st Qu.:20.12   1st Qu.:2.000   1st Qu.:0.000   347082  :  7  
              Median :28.00   Median :3.000   Median :0.000   CA. 2343:  7  
              Mean   :29.70   Mean   :2.309   Mean   :0.523   3101295 :  6  
              3rd Qu.:38.00   3rd Qu.:3.000   3rd Qu.:1.000   347088  :  6  
              Max.   :80.00   Max.   :3.000   Max.   :8.000   CA 2144 :  6  
              NA's   :177                                     (Other) :852  

## 1.2 Основные операции над таблицами и dplyr